In [2]:
##After installing, restart the kernel
##The following code works for Jupyter Lab
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Interactive residuals panel with TWO-WAY linking (top <-> bottom)
## Try to select data from the bottom figure and the top figure will be highlighted
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import statsmodels.api as sm
from sklearn.datasets import load_diabetes

# If you're in classic Jupyter or VS Code:
pio.renderers.default = "notebook_connected"  # try "vscode" or "jupyterlab" if needed

# --- Data & model
diab = load_diabetes()
df = pd.DataFrame(diab.data, columns=diab.feature_names)
df["target"] = diab.target

pred = "bmi"
X = sm.add_constant(df[[pred]])
y = df["target"]
m = sm.OLS(y, X).fit()
df["fitted"] = m.predict(X)
df["resid"]  = y - df["fitted"]

# --- Build FigureWidget with two linked charts
base = make_subplots(
    rows=2, cols=1, shared_xaxes=False,
    subplot_titles=(f"Actual vs Fitted (predictor: {pred})", "Residuals vs Fitted"),
    vertical_spacing=0.12
)
fig = go.FigureWidget(base)
fig.update_layout(height=650, width=800, dragmode="select", title="Linked Residuals Panel")

# Top: Actual vs Fitted (selectable)
top_scatter = go.Scatter(
    x=df["fitted"], y=df["target"],
    mode="markers", name="Actual vs Fitted",
    marker=dict(size=7, opacity=0.8),
    selected=dict(marker=dict(color="crimson", size=9)),
    unselected=dict(marker=dict(opacity=0.2)),
)

diag_line = go.Scatter(
    x=[df["fitted"].min(), df["fitted"].max()],
    y=[df["fitted"].min(), df["fitted"].max()],
    mode="lines", name="y = x", line=dict(dash="dash")
)

# Bottom: Residuals vs Fitted (selectable)
bottom_scatter = go.Scatter(
    x=df["fitted"], y=df["resid"],
    mode="markers", name="Residuals",
    marker=dict(size=7, opacity=0.8),
    selected=dict(marker=dict(color="crimson", size=9)),
    unselected=dict(marker=dict(opacity=0.2)),
)

fig.add_trace(top_scatter, 1, 1)
fig.add_trace(diag_line, 1, 1)
fig.add_trace(bottom_scatter, 2, 1)

fig.update_xaxes(title_text="Fitted (ŷ)", row=1, col=1)
fig.update_yaxes(title_text="Actual (y)", row=1, col=1)
fig.update_xaxes(title_text="Fitted (ŷ)", row=2, col=1)
fig.update_yaxes(title_text="Residual (y − ŷ)", row=2, col=1)
fig.add_hline(y=0, line_dash="dash", row=2, col=1)

# --- Two-way linking handlers
top = fig.data[0]      # top_scatter
bottom = fig.data[2]   # bottom_scatter

def link_top_to_bottom(trace, points, selector):
    # Select in TOP -> highlight same indices in BOTTOM
    inds = points.point_inds or None  # None clears selection
    bottom.selectedpoints = inds

def link_bottom_to_top(trace, points, selector):
    # Select in BOTTOM -> highlight same indices in TOP
    inds = points.point_inds or None
    top.selectedpoints = inds

top.on_selection(link_top_to_bottom)
bottom.on_selection(link_bottom_to_top)

# Optional: clear the other plot’s selection when user double-clicks background
def clear_on_deselect(layout, relayout):
    # Double-click often changes axis range; clear selections
    top.selectedpoints = None
    bottom.selectedpoints = None

fig.layout.on_change(clear_on_deselect, "xaxis", "yaxis", "xaxis2", "yaxis2")

fig  # In Jupyter, simply put `fig` as the last line to display


FigureWidget({
    'data': [{'marker': {'opacity': 0.8, 'size': 7},
              'mode': 'markers',
              'name': 'Actual vs Fitted',
              'selected': {'marker': {'color': 'crimson', 'size': 9}},
              'type': 'scatter',
              'uid': '9eb596e5-cf7d-43dc-9524-e1b99c98c202',
              'unselected': {'marker': {'opacity': 0.2}},
              'x': {'bdata': ('Bcy+obhWakAVsFTPx9BZQCA5a/rISm' ... 'YwAiFhQDgbIRYOp2dA2sADrfCyVEA='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAADgYkAAAAAAAMBSQAAAAAAAoG' ... 'AAAIBgQAAAAAAAgGtAAAAAAACATEA='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'line': {'dash': 'dash'},
              'mode': 'lines',
              'name': 'y = x',
              'type': 'scatter',
              'uid': 'b2eca63a-9e18-4396-b94e-ebcc7ade8d55',
              'x': [66.42293509524849, 314.0646295516874],
              'xaxis': 'x',
              'y':